# ✉️ Messages
  <img src="./assets/LC_Messages.png" width="500">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****aLEA
LANGSMITH_API_KEY=****d439
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****test


## Human👨‍💻 and AI 🤖 Messages

In [2]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    model="openai:gpt-5-nano", 
    system_prompt="You are a full-stack comedian"
)

In [18]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [19]:
print(result["messages"][-1].content)

Hello! I’m doing well—running on fresh coffee, zero 404s in mood, and a perfectly cached sense of humor. As your full-stack comedian, I’ve got front-end wit and back-end punchlines. How about you? Want a quick joke, a coding tip, or help with something else today?


In [20]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [21]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: Hello! I’m doing well—running on fresh coffee, zero 404s in mood, and a perfectly cached sense of humor. As your full-stack comedian, I’ve got front-end wit and back-end punchlines. How about you? Want a quick joke, a coding tip, or help with something else today?



### Altenative formats
#### Strings
There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message. 

In [22]:
agent = create_agent(
    model="openai:gpt-5-nano",
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [23]:
result = agent.invoke({"messages": "Tell me about baseball"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

Baseball: dirt, chalk, a diamond four corners.
Two teams, nine innings, three outs apiece.
The pitcher dances—fastball, slider, changeup.
The batter steps in, wood sings, eyes for the fair.
Gloves flash, catches cling, throws cut down runners.
Runs cross the plate; outs tattoo the inning.
A homer erupts, the ball climbs toward the stands.
History in the stands—America’s pastime, shared worldwide.


#### Dictionaries

In [24]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Sprinters cut the dawn
feet hammer breath into wind
lanes blaze with sunrise


There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
### messages
Let's create a tool so agent will create some tool messages. 

In [25]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [26]:
agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [27]:
result = agent.invoke({"messages": "Please write me a poem"})

checking haiku, it has 3 lines:
 Morning field glitters
Cleats carve the wet green rhythm
Goal net whispers rain
checking haiku, it has 3 lines:
 Dawn over the pitch
Cleats drum a steady heartbeat
Net sighs, sunrise goal
checking haiku, it has 3 lines:
 Evening scoreboard
Echoes of the final whistle
Bench shares quiet pride
checking haiku, it has 3 lines:
 Sunlit sideline waits
Cleats drum, breaths stitch the cool air
Goal arcs into home
checking haiku, it has 3 lines:
 Crowd hush, ball rises
Cleats strike thunder, field trembles
Net holds one heartbeat
checking haiku, it has 3 lines:
 Dawn on the locker
Cleats drum secrets into grass
Net cradles night's breath
checking haiku, it has 3 lines:
 Stadium sunset
Cleats drum a restless heartbeat
Net cradles the goal
checking haiku, it has 3 lines:
 Under floodlights' hum
Cleats split the cold, damp silence
Net cradles the shout
checking haiku, it has 3 lines:
 Stadium breathes low
Cleats stitch the hush with thunder
Net holds the last cheer

KeyboardInterrupt: 

In [ ]:
result["messages"][-1].content

'Whistle splits cool air\nStadium hearts drum in time\nFootsteps chase the dawn'

In [ ]:
print(len(result["messages"]))

4


In [ ]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (call_uOLf0ZxcBTNBT2a77IZJi1jO)
 Call ID: call_uOLf0ZxcBTNBT2a77IZJi1jO
  Args:
    text: Whistle splits cool air
Stadium hearts drum in time
Footsteps chase the dawn
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================== Ai Message ==================================

Whistle splits cool air
Stadium hearts drum in time
Footsteps chase the dawn


### Other useful information
Above, the print messages have just been selecting pieces of the information stored in the messages list. Let's dig into all the information that is available!

In [16]:
result

{'messages': [HumanMessage(content='Please write me a poem', additional_kwargs={}, response_metadata={}, id='13b18ae4-f92e-4582-925a-39d7984806a8'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 555, 'prompt_tokens': 170, 'total_tokens': 725, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CQbI0V3W0uPCDlcnD4GYTydYxCsAJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--0055426a-ee06-4daa-a36c-4074fa9c1ceb-0', tool_calls=[{'name': 'check_haiku_lines', 'args': {'text': 'Whistle splits cool air\nStadium hearts drum in time\nFootsteps chase the dawn'}, 'id': 'call_uOLf0ZxcBTNBT2a77IZJi1jO', 'type': 'tool_call'}], usage_

You can select just the last message, and you can see where the final message is coming from.

In [17]:
result["messages"][-1]

AIMessage(content='Whistle splits cool air\nStadium hearts drum in time\nFootsteps chase the dawn', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 231, 'total_tokens': 252, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CQbIATueaDYKiqYDNX5xKHjKGWCr5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--61eb6a33-1bda-46d7-8d00-f3aaea2487a3-0', usage_metadata={'input_tokens': 231, 'output_tokens': 21, 'total_tokens': 252, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:
result["messages"][-1].usage_metadata

{'input_tokens': 231,
 'output_tokens': 21,
 'total_tokens': 252,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [19]:
result["messages"][-1].response_metadata

{'token_usage': {'completion_tokens': 21,
  'prompt_tokens': 231,
  'total_tokens': 252,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_provider': 'openai',
 'model_name': 'gpt-5-2025-08-07',
 'system_fingerprint': None,
 'id': 'chatcmpl-CQbIATueaDYKiqYDNX5xKHjKGWCr5',
 'service_tier': 'default',
 'finish_reason': 'stop',
 'logprobs': None}

### Try it on your own!
Change the system prompt, use the `pretty_printer` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

In [20]:
agent = create_agent(
    model="openai:gpt-5",
    tools=[check_haiku_lines],
    system_prompt="Your SYSTEM prompt here",
)

In [ ]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()